
# **Cuaderno de Proyecto — Ciencia de Datos con YouTube**
**Curso:** SINT-200  
**Profesor:** Dr. Tomás de Camino Beck  
**Estudiante(s):** _Bernal Rojas Villalobos_  
**Fecha de entrega:** 21 de Octubre

---

## Instrucciones Generales

Reto: Exportar datos de tu actividad YouTube (Google Takeout), construir una matriz usuario-contenido con señales (vistas, likes, tiempo, etc.), hacer EDA de sesgos/“burbujas”, y entrenar dos recomendadores (colaborativo vs. basado en contenido). Comparar métricas (cosas como precision@k, recall@k, cobertura) y explicar errores.  



Este cuaderno sirve como **especificación y entregable** del proyecto. Debes completar cada sección marcada con **TODO** y dejar celdas de código **ejecutables** y **reproducibles**. El reto tiene dos proyectos:

1. **Proyecto 1 — Tu Huella YouTube: Recomendador y Análisis de Burbuja Algorítmica.**  
2. **Proyecto 2 — Detección de “Doomscrolling”: Predicción de sesiones extendidas.**

### Ética y Privacidad de Datos
- Puedes **anonimizar** tu información de YouTube (IDs, títulos, canales, tiempos) antes de subirla aquí.  
- Alternativamente, puedes usar datos de otra persona **con su consentimiento informado** y **anonimizados**.  
- No incluyas PII (información personal identificable) ni material sensible.  
- Incluye un **Anexo de Privacidad** explicando qué datos usaste, cómo los obtuviste y cómo los protegiste.

### Entregables
- Este **cuaderno de Colab** completo y ejecutable.  
- Carpeta `data/` con **muestras** de los datos (o datos sintéticos/anonimizados).  
- **Diccionario de datos** (descripción de campos, tipos, unidades, supuestos).  
- **Resultados y visualizaciones** dentro del notebook.  
- **Conclusiones** + **Recomendaciones** (acciones sugeridas) + **Limitaciones** + **Trabajo futuro**.
- Repositorio con estructura mínima:  
  ```
  README.md
  data/        # muestras o datos anonimizados
  notebooks/   # este cuaderno
  src/         # funciones reutilizables
  reports/     # figuras / tablas clave
  ```

### Rúbrica (100 pts)
- **Charter/Problema y utilidad (10 pts)**: objetivos claros, hipótesis, valor para el usuario.  
- **Adquisición y calidad de datos (10 pts)**: trazabilidad, permisos, limpieza básica.  
- **EDA y visualizaciones (20 pts)**: distribución, outliers, correlaciones, sesgos/segmentos.  
- **Baselines y metodología (10 pts)**: definición de referencia simple y por qué.  
- **Modelado (20 pts)**: al menos **2 enfoques** comparados, justificación.  
- **Evaluación (15 pts)**: métricas adecuadas, validación (temporal cuando aplique), error analysis.  
- **Reproducibilidad (5 pts)**: semillas, funciones, estructura clara.  
- **Conclusiones & ética (10 pts)**: hallazgos accionables y reflexiones de privacidad/sesgo.

---



## 0. Preparación del entorno (ejecutar una vez)


In [1]:
# Clonar tu repositorio
!git clone https://github.com/brojas7/AnaliticaHistorialYoutube.git

# Ir al directorio del proyecto
%cd AnaliticaHistorialYoutube

# TODO: Ajusta versiones si lo necesitas. Evita dependencias innecesarias.
!pip -q install pandas numpy matplotlib scikit-learn textblob python-dateutil tqdm dateparser google-generativeai  gensim

Cloning into 'AnaliticaHistorialYoutube'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 122 (delta 4), reused 0 (delta 0), pack-reused 110 (from 1)
Receiving objects: 100% (122/122), 45.05 MiB | 18.99 MiB/s, done.
Resolving deltas: 100% (35/35), done.
/content/AnaliticaHistorialYoutube
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 36.9 MB/s eta 0:00:00


In [2]:
# Imports base y configuración
import os, json, math, random, itertools, collections, gzip, re, string, time, zipfile, io
from datetime import datetime, timedelta
from dateutil import parser as dateparser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, mean_absolute_error, mean_squared_error
)

import sys
sys.path.append('/content/AnaliticaHistorialYoutube/src')

from youtube_utils import load_watch_history, anonymize_df, sessionize

# Reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

pd.set_option("display.max_columns", 120)
pd.set_option("display.max_rows", 200)

print("Entorno listo. Versión de pandas:", pd.__version__)


Entorno listo. Versión de pandas: 2.2.2



## 1. Anexo de Privacidad y Origen de Datos (obligatorio)
**TODO:** Explicar:
- Fuente de datos (Google Takeout, exportación manual, datos de tercero con consentimiento, etc.).  
- Estrategia de **anonimización** (por ejemplo: hashing de IDs/URLs, truncado de timestamps, agrupación por hora/día).  
- Contenido eliminado o agregado (p. ej., datos sintéticos para completar campos).  
- Limitaciones y riesgos residuales.



## 2. Selección de Proyecto
**Marca con una X**

- [ ] **Proyecto 1 — Recomendador YouTube & Burbuja Algorítmica**  
- [ ] **Proyecto 2 — Detección de Doomscrolling (clasificación temporal)**



## 3. Utilidades comunes para YouTube (ingesta y parsing)

Para **Proyecto 1** y **Proyecto 2** puedes usar datos de **Google Takeout**:  
- `watch-history.json` (o `watch-history.html` en exportaciones antiguas)  
- `search-history.json` (opcional)  
- `likes.csv` / `subscriptions.csv` (según disponibilidad)

> **Nota:** Los formatos de Takeout pueden cambiar con el tiempo. Ajusta el parser según tu exportación.



# **Limpieza, Enriquecimiento y Transformación del dataset**


In [3]:
# Ejemplo de uso
df_search = load_watch_history('data/historial-de-búsqueda.json')
df_watch = load_watch_history('data/historial-de-reproducciones.json')
df_suscipciones = pd.read_csv('data/suscripciones.csv')

#df = anonymize_df(df)
#df = sessionize(df)
print(df_search.shape)
print(df_watch.shape)
print("Eventos:", len(df_watch), "Rango:", df_watch['timestamp'].min(), "->", df_watch['timestamp'].max())

(12090, 6)
(36842, 6)
Eventos: 36842 Rango: 2018-02-22 02:05:35.427000+00:00 -> 2025-10-18 01:02:57.606000+00:00


In [4]:
# Copia base
df_watch_clean = df_watch.copy()

# Limpieza del título (remover "Has visto")
df_watch_clean['video_title'] = df_watch_clean['title'].str.replace(r'^Has visto\s+', '', regex=True)

# Convertir timestamps
df_watch_clean['watched_at'] = pd.to_datetime(df_watch_clean['timestamp'], utc=True)

# Variables derivadas
df_watch_clean['weekday'] = df_watch_clean['watched_at'].dt.day_name()
df_watch_clean['hour'] = df_watch_clean['watched_at'].dt.hour
df_watch_clean['hour_group'] = pd.cut(df_watch_clean['hour'],
                                      bins=[-1,6,12,18,24],
                                      labels=['madrugada','mañana','tarde','noche'])

In [5]:
df_suscripciones_clean = df_suscipciones.rename(columns={
    "Título del canal": "channel_title",
    "URL del canal": "channel_url",
    "ID del canal": "channel_id"
})

In [6]:
df_search_clean = df_search.copy()
df_search_clean['timestamp'] = pd.to_datetime(df_search_clean['timestamp'], utc=True)
df_search_clean['search_terms'] = df_search_clean['title'].str.replace('Buscaste', '').str.strip()

Enlace entre vistas y suscripciones

In [7]:
# Renombramos la columna 'channel' en el historial para igualarla con suscripciones
df_watch_clean = df_watch_clean.rename(columns={'channel': 'channel_title'})

df_main = df_watch_clean.merge(
    df_suscripciones_clean[['channel_title', 'channel_id']],
    on='channel_title',
    how='left',
    indicator=True
)

df_main['is_subscribed'] = (df_main['_merge'] == 'both').astype(int)
df_main.drop(columns=['_merge'], inplace=True)

3️Señales de interacción (para matriz usuario–contenido)

In [ ]:
df_main['interaction_score'] = 1.0
df_main.loc[df_main['is_subscribed'] == 1, 'interaction_score'] += 0.5

Valores nulos en canal
son anuncios i videos borrados

In [ ]:
df_main.shape

(36842, 14)

In [ ]:
df_main[df_main['channel_title'].isna()][['video_title', 'url']].head(10)


,video_title,url
3,https://www.youtube.com/watch?v=l8dasJTQdw8,https://www.youtube.com/watch?v=l8dasJTQdw8
9,https://www.youtube.com/watch?v=Qh8vqGj_Dbg,https://www.youtube.com/watch?v=Qh8vqGj_Dbg
16,https://www.youtube.com/watch?v=vxDjs9wQrhw,https://www.youtube.com/watch?v=vxDjs9wQrhw
21,https://www.youtube.com/watch?v=09wmbNmZsbc,https://www.youtube.com/watch?v=09wmbNmZsbc
33,https://www.youtube.com/watch?v=or6p6659Jrg,https://www.youtube.com/watch?v=or6p6659Jrg
75,https://www.youtube.com/watch?v=vxDjs9wQrhw,https://www.youtube.com/watch?v=vxDjs9wQrhw
77,https://www.youtube.com/watch?v=UZwXbecjg1Y,https://www.youtube.com/watch?v=UZwXbecjg1Y
81,https://www.youtube.com/watch?v=09wmbNmZsbc,https://www.youtube.com/watch?v=09wmbNmZsbc
97,https://www.youtube.com/watch?v=EM9OG65ccVI,https://www.youtube.com/watch?v=EM9OG65ccVI
185,https://www.youtube.com/watch?v=educpLKQbEg,https://www.youtube.com/watch?v=educpLKQbEg


In [ ]:
mask_valid = df_main['channel_title'].notna() & ~df_main['video_title'].str.contains("anuncio", case=False, na=False)
df_main_clean = df_main[mask_valid].copy()

In [ ]:
df_ads = df_main[~mask_valid]
df_ads.to_csv("data/df_ads_removed.csv", index=False)

ANON


# **Proyecto 1 — Tu Huella YouTube: Recomendador & Burbuja Algorítmica**

### Objetivo
1) Construir **dos recomendadores** con tus datos de visualización:  
   - **Baseline de popularidad** (o popularidad por canal/categoría).  
   - **Modelo basado en contenido** (TF‑IDF/embeddings por título/canal) **o** **colaborativo** (si tienes interacciones de múltiples usuarios/fuentes).  
2) Medir **Precision@k, Recall@k y Coverage** (y *diversidad*) en un esquema **offline**.  
3) Analizar posibles **sesgos o “burbujas”** (temas/canales dominantes por hora, día, duración).  

### Requisitos mínimos
- **EDA**: distribución de vistas por canal, hora del día, día de semana, duración de sesiones, *top‑k* temas.  
- **Ingeniería de features** (ej.: tokenización títulos, lematización opcional, normalización de canales).  
- **Comparación de al menos 2 enfoques** de recomendación.  
- **Evaluación offline** con *train/test split temporal*.  
- **Análisis de errores** y discusión de sesgos/limitaciones.

---

## 4. Charter del Proyecto 1 (llenar)
**TODO:** Define el propósito, preguntas clave y utilidad (qué decisiones permitirán tus hallazgos).



## 5. Carga de datos (Proyecto 1)
**TODO:** Sube tu `watch-history.json` (anonimizado si aplica) a `data/` y cárgalo.


In [8]:
df_main = pd.read_csv("data/df_enrich_enriquecido.csv")
print(df_main.shape)
df_main.head()

(31217, 18)


,timestamp,title,channel_title,channel_id_x,video_id,url,video_title,watched_at,weekday,hour,hour_group,channel_id_y,is_subscribed,interaction_score,category,subtopic,format,keywords
0,2018-02-22 02:05:35.427000+00:00,Has visto La canción más hermosa en piano fáci...,Paula Yessenia Barragan Izquierdo,NaN,ro1rC9dL5EQ,https://www.youtube.com/watch?v=ro1rC9dL5EQ,La canción más hermosa en piano fácil de aprender,2018-02-22 02:05:35.427000+00:00,Thursday,2,madrugada,NaN,0,1.0,Tutorial,Music Lesson,Tutorial,"['piano', 'easy to learn', 'music lesson']"
1,2018-02-22 03:49:54.096000+00:00,Has visto Ed Sheeran - Perfect - EASY Piano Tu...,Peter PlutaX,NaN,p1WCR7vNcIw,https://www.youtube.com/watch?v=p1WCR7vNcIw,Ed Sheeran - Perfect - EASY Piano Tutorial by ...,2018-02-22 03:49:54.096000+00:00,Thursday,3,madrugada,NaN,0,1.0,Tutorial,Music Lesson,Tutorial,"['Ed Sheeran', 'Perfect', 'piano tutorial', 'e..."
2,2018-02-22 05:27:06.367000+00:00,Has visto How I proposed: fairytale story with...,bigchewypretzels,NaN,Srw9gXDa24g,https://www.youtube.com/watch?v=Srw9gXDa24g,How I proposed: fairytale story with puzzles a...,2018-02-22 05:27:06.367000+00:00,Thursday,5,madrugada,NaN,0,1.0,Entertainment,Relationship/Life Event,Vlog/Storytelling,"['proposal', 'love story', 'fairytale', 'relat..."
3,2018-02-22 06:26:38.438000+00:00,Has visto TOP 5 BROMAS - Bromas para hacer a t...,BROMAS Y MÁS TVOAQUI,NaN,Q4UilByqMTc,https://www.youtube.com/watch?v=Q4UilByqMTc,"TOP 5 BROMAS - Bromas para hacer a tus amigos,...",2018-02-22 06:26:38.438000+00:00,Thursday,6,madrugada,NaN,0,1.0,Entertainment,Comedy/Prank,Prank Compilation,"['pranks', 'friends', ""Valentine's Day"", 'come..."
4,2018-02-22 14:20:32.963000+00:00,Has visto Morat - Yo Más Te Adoro,MoratVEVO,NaN,pqJBXjzBr_U,https://www.youtube.com/watch?v=pqJBXjzBr_U,Morat - Yo Más Te Adoro,2018-02-22 14:20:32.963000+00:00,Thursday,14,tarde,NaN,0,1.0,Music,Music Performance,Official Music Video,"['Morat', 'Yo Más Te Adoro', 'Latin Pop', 'Off..."


In [9]:
import hashlib
import pandas as pd
import numpy as np

def hash_value(x):
    """Aplica hash SHA-256 truncado (12 chars) a cualquier valor."""
    if pd.isna(x):
        return np.nan
    return hashlib.sha256(str(x).encode()).hexdigest()[:12]

# Copia del dataframe original
df_hashed = df_main.copy()

# Aplica el hash a todo el dataframe
df_hashed = df_hashed.applymap(hash_value)

# Verifica resultado
df_hashed.head()

/tmp/ipython-input-359848926.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_hashed = df_hashed.applymap(hash_value)


,timestamp,title,channel_title,channel_id_x,video_id,url,video_title,watched_at,weekday,hour,hour_group,channel_id_y,is_subscribed,interaction_score,category,subtopic,format,keywords
0,43b624cafb0c,b5c2fe6c821d,561cc3b6b7fb,NaN,d8b7eb145254,5b328611dfd1,7f78ca26e123,43b624cafb0c,fc2662062ffd,d4735e3a265e,c420fdb9ac10,NaN,5feceb66ffc8,d0ff5974b6aa,c26982b1425d,1ecbd27cfc92,c26982b1425d,1dc189d46c1d
1,ac49439f1641,9267aa5a5ce2,bc5af354f824,NaN,7e655fd0d398,2aed642c7f9d,11eaa6ac0ea2,ac49439f1641,fc2662062ffd,4e07408562be,c420fdb9ac10,NaN,5feceb66ffc8,d0ff5974b6aa,c26982b1425d,1ecbd27cfc92,c26982b1425d,405ab863bc17
2,aa80c41676a7,5035d49868e3,07254e8fb92b,NaN,585412c8f79b,3688faaf95fb,d954a349049f,aa80c41676a7,fc2662062ffd,ef2d127de37b,c420fdb9ac10,NaN,5feceb66ffc8,d0ff5974b6aa,ceaa553e838f,4fae335077e6,200fafff5b00,c75c87529646
3,d6273362f33c,bc92ec09f80c,b03d217c9070,NaN,ad81fdd792ac,0d89a0111957,77d86ffa77c6,d6273362f33c,fc2662062ffd,e7f6c011776e,c420fdb9ac10,NaN,5feceb66ffc8,d0ff5974b6aa,ceaa553e838f,d63b6b556ce7,ff14d2076ecc,3edeeb3b1efd
4,52c804da8e4d,f0f24a0474ac,0f66b5b56794,NaN,4e7403885505,018b4f680689,c13793508e6c,52c804da8e4d,fc2662062ffd,8527a891e224,dfc02909a308,NaN,5feceb66ffc8,d0ff5974b6aa,6eb00b4b2614,9423a442e22b,754dc3f10982,f642e0d4921f



## 6. EDA (Proyecto 1)
**TODO:** Explora sesgos por canal/tema/horario. Muestra tablas y gráficos clave.


In [10]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

[===========================================-------] 87.8% 1460.5/1662.8MB downloaded

In [11]:
import ast
import pandas as pd

def str_to_list(column):
    """
    Convierte una columna de strings con formato de lista en listas reales.
    Ejemplo: "['piano', 'easy to learn']" -> ['piano', 'easy to learn']

    Parámetros:
        column (pd.Series): Columna de un DataFrame.

    Retorna:
        pd.Series: Columna convertida a listas reales.
    """
    def safe_eval(x):
        if isinstance(x, str):
            try:
                return ast.literal_eval(x)
            except Exception:
                return []
        elif isinstance(x, list):
            return x
        else:
            return []

    return column.apply(safe_eval)

In [12]:
# Ejemplo con tu DataFrame
df_main["keywords"] = str_to_list(df_main["keywords"])

In [13]:
def get_text_vector(text, model):
    """
    Promedia los embeddings de las palabras de un texto (frase o lista).
    Devuelve vector 300D.
    """
    if isinstance(text, list):
        tokens = text
    elif isinstance(text, str):
        tokens = text.lower().split()
    else:
        tokens = []

    valid_vectors = [model[w] for w in tokens if w in model]
    if not valid_vectors:
        return np.zeros(model.vector_size)
    return np.mean(valid_vectors, axis=0)


def get_keyword_vectors(keyword_list, model):
    """
    Convierte lista de keywords (palabras o frases) en lista de vectores (300D).
    Tokeniza correctamente y maneja mayúsculas/minúsculas.
    """
    vectors = []
    if not isinstance(keyword_list, list):
        return []

    for kw in keyword_list:
        if not isinstance(kw, str):
            continue

        tokens = kw.split()  # divide frases tipo "easy to learn"
        token_vectors = []

        for token in tokens:
            # Buscar distintas capitalizaciones
            if token in model:
                token_vectors.append(model[token])
            elif token.capitalize() in model:
                token_vectors.append(model[token.capitalize()])
            elif token.upper() in model:
                token_vectors.append(model[token.upper()])

        # Promedia las palabras de la frase
        if token_vectors:
            vectors.append(np.mean(token_vectors, axis=0))
        else:
            vectors.append(np.zeros(model.vector_size))

    return vectors

# ============================================================
# 🔹 4️⃣ Generar embeddings por columna
# ============================================================

df_embed = df_main.copy()
cols_single_vector = ["category", "subtopic", "format", "video_title", "channel_title"]

print("Generando embeddings columna por columna...")

# Embeddings individuales (vector promedio 300D)
for col in cols_single_vector:
    print(f"→ Procesando {col}")
    df_embed[f"{col}_vec"] = df_embed[col].apply(lambda x: get_text_vector(x, model))

# Embeddings múltiples (lista de vectores 300D por keyword)
print("→ Procesando keywords (lista de vectores)...")
df_embed["keywords_vec"] = df_embed["keywords"].apply(lambda x: get_keyword_vectors(x, model))

print("✅ Embeddings generados exitosamente.")

# ============================================================
# 🔹 5️⃣ Verificación de salida
# ============================================================
cols_to_show = [
    "keywords",
    "category_vec",
    "subtopic_vec",
    "format_vec",
    "video_title_vec",
    "channel_title_vec",
    "keywords_vec"
]

print("\n🔍 Ejemplo de salida:")
display(df_embed[cols_to_show].head(5))

In [14]:
#df_embed.loc[1, "keywords_vec"][1]

In [15]:
del model


## 7. Partición temporal y definición de tareas (Proyecto 1)
**TODO:** Define ventana de entrenamiento y de evaluación para recomendación **offline**.


In [16]:
# Asegurar que timestamp es datetime
df_embed["timestamp"] = pd.to_datetime(df_embed["timestamp"], errors="coerce")

# Verifica que la conversión haya funcionado
print(df_embed["timestamp"].dtypes)
print(df_embed["timestamp"].head())

In [17]:
# Split temporal: por ejemplo, último 20% del tiempo como test
cut_ts = df_embed['timestamp'].quantile(0.8)
train = df_embed[df_embed['timestamp'] <= cut_ts].copy()
test  = df_embed[df_embed['timestamp'] >  cut_ts].copy()

print("train:", train['timestamp'].min(), "->", train['timestamp'].max(), "n=", len(train))
print("test :", test['timestamp'].min(),  "->", test['timestamp'].max(),  "n=", len(test))


## 8. Baseline de popularidad (Proyecto 1)
Genera recomendaciones **sin personalización** como referencia.


In [18]:

# Top-N por popularidad (baseline)
K = 10  # tamaño de recomendación
top_items = train['video_id'].value_counts().head(100).index.tolist()

def recommend_popularity(k=K):
    return top_items[:k]

# Conjunto de ítems verdaderos en test (lo visto en test)
true_items = set(test['video_id'].dropna().unique().tolist())

def precision_at_k(recommended, true_set):
    if len(recommended) == 0: return 0.0
    hit = sum(1 for x in recommended if x in true_set)
    return hit / len(recommended)

def recall_at_k(recommended, true_set):
    if len(true_set) == 0: return 0.0
    hit = sum(1 for x in recommended if x in true_set)
    return hit / len(true_set)

# Eval baseline
rec = recommend_popularity(K)
p = precision_at_k(rec, true_items)
r = recall_at_k(rec, true_items)
coverage = len(set(top_items)) / max(1, df_embed['video_id'].nunique())

print(f"Baseline Popularidad -> P@{K}={p:.3f}  R@{K}={r:.3f}  Cobertura={coverage:.3f}")


## 9. Recomendador basado en contenido **(ejemplo TF-IDF por título/canal)**
**TODO:** Implementa TF‑IDF (o embeddings) y calcula similitud contenido‑a‑contenido para recomendar.


In [ ]:
# ============================================================
# 🔹 1️⃣ Importar librerías
# ============================================================
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# ============================================================
# 🔹 2️⃣ Crear vector de contenido global por video
# ============================================================
def get_global_embedding(row, weight_keywords=2.0):
    """
    Combina todos los embeddings (category, subtopic, format, title, channel, keywords)
    en un solo vector promedio ponderado.
    """
    vectors = []

    # columnas simples
    for col in ["category_vec", "subtopic_vec", "format_vec", "video_title_vec", "channel_title_vec"]:
        if isinstance(row[col], np.ndarray) and row[col].any():
            vectors.append(row[col])

    # keywords (lista de vectores)
    if isinstance(row["keywords_vec"], list) and len(row["keywords_vec"]) > 0:
        kw_mean = np.mean(row["keywords_vec"], axis=0)
        vectors.append(kw_mean * weight_keywords)  # ponderamos keywords un poco más

    if len(vectors) == 0:
        return np.zeros(300)

    # promedio de todos los embeddings
    return np.mean(vectors, axis=0)

df_embed["content_vec"] = df_embed.apply(lambda r: get_global_embedding(r), axis=1)

print("✅ Embeddings globales creados:", df_embed["content_vec"].iloc[0].shape)

# ============================================================
# 🔹 3️⃣ Matriz de similitud de coseno
# ============================================================
# Creamos una matriz [n_videos x 300]
matrix = np.vstack(df_embed["content_vec"].values)
matrix = normalize(matrix)  # normalizamos para estabilidad numérica

# Matriz de similitud coseno
cosine_sim = cosine_similarity(matrix)

print("✅ Matriz de similitud creada:", cosine_sim.shape)

# ============================================================
# 🔹 4️⃣ Función para recomendar
# ============================================================
def recommend(video_id, df, sim_matrix, top_k=5):
    """
    Retorna los videos más similares al video dado (por índice).
    """
    if video_id >= len(df):
        raise ValueError("El índice de video está fuera de rango.")

    # Vector de similitudes para el video dado
    sim_scores = list(enumerate(sim_matrix[video_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Excluir el propio video y tomar los top_k más similares
    top_indices = [i for i, s in sim_scores[1:top_k+1]]

    return df.iloc[top_indices][["video_title", "category", "subtopic", "channel_title"]]

In [ ]:
# ============================================================
# 🔹 5️⃣ Ejemplo de uso
# ============================================================
video_id = 10  # puedes cambiarlo
print(f"\n🎬 Video base: {df_embed.loc[video_id, 'video_title']}")
print("🔎 Recomendaciones similares:")
display(recommend(video_id, df_embed, cosine_sim, top_k=5))

In [107]:

# EJEMPLO ESQUELETO (completa con TF-IDF real si lo deseas)
# Aquí usamos una heurística mínima por canal/tokens de título para ilustrar el flujo.

from collections import Counter

def tokenize_title(s):
    if pd.isna(s): return []
    s = s.lower()
    s = re.sub(r"[^a-z0-9áéíóúüñ\s]", " ", s)
    tok = [t for t in s.split() if len(t) > 2]
    return tok

train = train.copy()
train['tokens'] = train['title'].apply(tokenize_title)

# "Perfil" de intereses por tokens (muy básico)
profile = Counter(itertools.chain.from_iterable(train['tokens'].tolist()))
top_terms = [t for t, _ in profile.most_common(50)]

def recommend_content_based(k=10):
    # Recomienda ítems del set de entrenamiento por coincidencia con términos del perfil
    scores = []
    for vid, grp in train.groupby('video_id'):
        toks = list(itertools.chain.from_iterable(grp['tokens'].tolist()))
        score = sum(1 for t in toks if t in top_terms)
        scores.append((vid, score))
    scores.sort(key=lambda x: x[1], reverse=True)
    return [vid for vid, s in scores[:k]]

rec_cb = recommend_content_based(K)
p_cb = precision_at_k(rec_cb, true_items)
r_cb = recall_at_k(rec_cb, true_items)
cov_cb = len(set(train['video_id'])) / max(1, df_watch['video_id'].nunique())
print(f"Contenido (heurístico) -> P@{K}={p_cb:.3f}  R@{K}={r_cb:.3f}  Cobertura={cov_cb:.3f}")

# TODO: Sustituir por TF-IDF/embeddings reales y mejorar evaluación (por usuario/ventanas).


NameError: name 'df_watch' is not defined


## 10. Análisis de burbuja/sesgo (Proyecto 1)
**TODO:** Mide concentración por canal/tema, horarios de consumo, diversidad de recomendaciones.
Propón **intervenciones** para aumentar diversidad sin perder pertinencia.



## 11. Conclusiones y trabajo futuro (Proyecto 1)
**TODO:** Resume hallazgos, limitaciones y siguientes pasos.
